In [2]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install -U langchain-community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.8/166.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=2740fef1223bc5de9e8499dde179b9b3c11748918ee62f9ce846492dd4f287ef
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4b

In [195]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import BaseRetriever
from langchain import PromptTemplate

class CustomQA():

    """
    custom QA chain.
    Checks if the retrriever is able to match any docs.
    If so, the LLM is called, else, No LLM is called.
    A message is returned.
    """
    def __init__(self, llm_model, retriever: BaseRetriever):


        template = """
          System: You are a intelligent question answering bot, that provides short answers to questions.

          Instructions:
          1. Make Use the following context to to answer the user query.
          2. Make sure the answer is short and brief.
          3. Use the context to generate the answer.

          Context: {context}

          Human: {inputs}

          Assistant:"""

        prompt_template = PromptTemplate(
              input_variables=["inputs", "context"],
              template=template,
        )


        self.qa_chain = create_stuff_documents_chain(llm=llm_model, prompt = prompt_template)
        self.retriever = retriever
        self.llm_model = llm_model

    def run(self, inputs):

        question = inputs["query"]
        retrieved_docs = self.retriever.get_relevant_documents(question)

        if not retrieved_docs:
            print("No matching documents found. Skipping LLM call.")
            return {"result": "I couldn't find any information about that.", "source_documents": []}
        else:
            asnwer = self.qa_chain.invoke({
                "inputs": question,
                "context": retrieved_docs
            })
            response = {}
            response["result"] = asnwer
            response["source_documents"] = retrieved_docs
            return response



In [196]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, UnstructuredMarkdownLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI


import os

os.environ["OPENAI_API_KEY"] = "<< -- OPEN AI KEY -- >>"


class RAG:
  def __init__(self, doc_file, query_file):

    if not self.check_file_exists(doc_file):
      raise FileNotFoundError(f"File not found: {doc_file}")

    if not self.check_file_exists(query_file):
      raise FileNotFoundError(f"File not found: {query_file}")

    self.document = UnstructuredMarkdownLoader(doc_file).load()
    self.chunks = self.split_document(self.document)
    self.vector_store = self.create_vector_store(self.chunks)
    self.queries = load_json_file(query_file)
    self.results = []
    self.retriever = self.vector_store.as_retriever(search_type="similarity_score_threshold",
                                                    search_kwargs={"score_threshold": 0.2})
    self.llm = ChatOpenAI(model="gpt-4o")
    self.qa = CustomQA(llm_model=self.llm,
                                retriever=self.retriever)

  def check_file_exists(self, file_path):
    return os.path.exists(file_path)

  def split_document(self, document, chunk_size: int = 500, chunk_overlap: int = 0):
      """
      takes in the .MD file and splits it into chunks of text.
      input:
        file_path: string = path to the .MD file.
        chunk_size: integer = Number of chars in a single chunk.
        chunk_overlap: integer = Number of chars that are borrowed from the previous chunk.
      output:
        texts: list = list of chunks of text.

      """

      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size, chunk_overlap=chunk_overlap
      )
      texts = text_splitter.split_documents(document)
      return texts

  def create_vector_store(self, chunks):
      """
      takes in the chunks of text and creates a FAISS vector store.
      By default, the model used is all-MiniLM-L6-v2.
      input:
        chunks: list = list of chunks of text.
      output:
        db: FAISS = vector store.
      """
      embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
      db = FAISS.from_documents(chunks, embeddings)
      return db

  def get_answer(self, query):
      """
      takes in a query and returns the answer.
      input:
        query: string = query to be answered.
      output:
        result: JSON = anser and source documents.
      """
      result = self.qa.run({"query": query})
      source_docs = result["source_documents"]

      return result

  def load_json_file(self, file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in '{file_path}'.")
        return None

  def store_json_file(self):
    """
    stores the results in a JSON file
    stores a list of dictionaries in a JSON file.
    """

    with open("results.json", "w") as f:
        json.dump(self.results, f, indent=4)


  def run(self):
    """
    runs the RAG pipeline on the list of queries.
    stores the results in a JSON file.
    """

    results = []
    for query in self.queries:
        result = self.get_answer(query['text'])
        answer, source_documents = result["result"], result["source_documents"]
        query["answer"] = answer
        query["source_documents"] = [{"id": doc.id, "content": doc.page_content} for doc in source_documents]

        results.append(query)

    self.results = results
    self.store_json_file()
    return self.results




In [197]:
rag_agent = RAG("medicare_comparison.md","queries.json")


In [198]:
res = rag_agent.run()

In [174]:
for i in res:
  print(f"quesiton: {i['text']}")
  print(f"answer: {i['answer']}")
  print("**********************************")
  print("Source documents:")
  for j in i['source_documents']:
    print("---------------------------------------------")
    print(f"documentID: {j['id']}")
    print(f"page content: {j['content']}")
    print("\n")

  print("==============================================\n")

quesiton: Can I see any doctor with Original Medicare?
answer: Original Medicare

Original Medicare consists of Part A (Hospital Insurance) and Part B (Medical Insurance). It covers most medically necessary services and supplies in hospitals, doctors’ offices, and other healthcare facilities. However, it doesn’t cover certain services, such as routine physical exams, most dental care, and vision care. You have the option to join a separate Medicare drug plan (Part D) for prescription drug coverage. Additionally, you can purchase a Medigap policy to help cover some out-of-pocket costs, such as copayments, coinsurance, and deductibles.

Medicare Advantage

Medicare Advantage (Part C) plans are offered by private insurance companies approved by Medicare. These plans cover all services that Original Medicare covers, but they may also provide extra benefits, such as vision, dental, and hearing coverage, which Original Medicare does not cover. Unlike Original Medicare, Medicare Advantage pla

In [ ]:
# prompt: create a README for the project. Describe each component in detail. Include instructions to run the project

- OpenAI API Key: Set your OpenAI API key as an environment variable named `OPENAI_API_KEY`.

**Steps:**

1. **Prepare Input Files:**
   - Create a markdown file named `medicare_comparison.md` containing the document to be used for question answering.
   - Create a JSON file named `queries.json` with an array of objects, where each object has a "text" field representing the question to be answered.  Example:



In [179]:
!python app.py

/usr/local/lib/python3.11/dist-packages/langchain/__init__.py:30: UserWarning: Importing PromptTemplate from langchain root module is no longer supported. Please use langchain_core.prompts.PromptTemplate instead.
  warnings.warn(
/content/rag.py:4: LangChainDeprecationWarning: Importing TextLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import TextLoader

with new imports of:

>> from langchain_community.document_loaders import TextLoader
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.document_loaders import TextLoader, UnstructuredMarkdownLoader
/content/rag.py:4: LangChainDeprecationWarning: Importing UnstructuredMarkdownLoader from langchain.document_loaders is deprecated. Please replace deprecated imports:

>> from langchain.document_loaders import UnstructuredMarkdownLoader

with ne